In [3]:
import pandas as pd
import datetime
from datetime import datetime as dt
import calendar

### ------------------------- ingest rystad production -----------------------------

In [53]:
def rystad_datetime(df):
    df['date'] = pd.to_datetime(df['Year Month'])
    df.drop(columns = 'Year Month', inplace=True)
    df.set_index('date', inplace=True)
    return df

In [54]:
def clean_country_names(df):
    df['Country'] = df['Country'].str.lower()
    df['Country'] = df['Country'].replace(' ', '_', regex=True)
    return df

In [69]:
def loop_and_upload(df):
    country_list = df['Country'].unique()
    for c in country_list:
        country_df = df[df['Country']==c]
        print(country_df['Production (kbbl/d)'])
    return print(f'{len(country_list)} series uploaded')

In [70]:
filepath = '//petroineos.local/dfs/Department Shared Folders/~Analysis Department/crude/production/RystadProduction.xlsx'

In [71]:
rystad = clean_country_names(rystad_datetime(pd.read_excel(filepath)))

In [75]:
# loop_and_upload(rystad)

In [74]:
rystad['Country'].unique()

array(['turkey', 'ireland', 'united_kingdom', 'norway', 'denmark',
       'poland', 'germany', 'austria', 'france', 'netherlands', 'spain',
       'italy', 'greece', 'hungary', 'czech_republic', 'slovakia'],
      dtype=object)

### ------------------------- combined PI and rystad balances -----------------------------

In [ ]:
combined_data = pd.read_excel('PI_woodmac.xlsx',sheet_name='final', header=1)

In [ ]:
unpivoted = combined_data.melt(id_vars=['Country', 'code', 'Region', 'source'], var_name='M', value_name='runs_mbd')

In [ ]:
unpivoted['month'] = pd.to_datetime(unpivoted['M']) 
unpivoted.drop(columns='M', inplace=True)

In [ ]:
unpivoted['runs_kbd'] = unpivoted['runs_mbd']*1000
unpivoted.drop(columns='runs_mbd', inplace=True)

In [ ]:
len(unpivoted)

In [ ]:
country_production = pd.read_excel("rystad balances.xlsx",)
country_production['month'] = pd.to_datetime(country_production['month'])
country_production.drop(columns=['Year', 'Month'], inplace=True)
country_production.rename(columns={'Country':'Country_rystad'}, inplace=True)

In [ ]:
mapping = pd.read_excel('//petroineos.local/DFS/Profiles/syedahmad/Desktop/country mapping.xlsx', sheet_name='Rystad mapping')
mapping.drop(columns='balances region', inplace=True)

In [ ]:
production_cc = country_production.merge(mapping, how='left', on=['Country_rystad'])
len(production_cc)

In [ ]:
len(production_cc[production_cc.code.isnull()])

In [ ]:
production_cc = production_cc[production_cc.code.notnull()].copy()
len(production_cc)

------------------find duplicates-------------------------

In [ ]:
runs_duplicates = unpivoted[unpivoted.duplicated(subset=['code', 'month'], keep=False)]
runs_duplicates.head()

In [ ]:
prod_duplicates = production_cc[production_cc.duplicated(subset=['code', 'month'], keep=False)]
prod_duplicates.head()

In [ ]:
country_balance = unpivoted.merge(production_cc, how='inner', on=['code', 'month'])
len(country_balance)

In [ ]:
country_balance.head()

In [ ]:
country_balance.to_csv('country_balances.csv')

### --------------regional tables---------------

In [ ]:
regional_runs = unpivoted.groupby(['Region', 'month']).sum().reset_index()

In [ ]:
regional_runs.head()

In [ ]:
len(regional_runs)

In [ ]:
regional_production = pd.read_excel('rystad balances regional.xlsx', sheet_name='prod')

In [ ]:
regional_production['month'] = pd.to_datetime(regional_production['Month'])
regional_production.drop(columns='Month', inplace=True)

In [ ]:
regional_production.head()

In [ ]:
len(regional_production)

In [ ]:
regional_runs_production = regional_production.merge(regional_runs, how='left', on=['Region', 'month'])

In [ ]:
regional_runs_production['balance'] = regional_runs_production['Production (kbbl/d)']-regional_runs_production['runs_kbd']

In [ ]:
regional_runs_production.head()

In [ ]:
regional_runs_production.to_csv('global_regional_balance.csv')

## ------------------- JBC data ------------------

In [ ]:
jbc_mapping_raw = pd.read_excel('jbc countries.xlsx', sheet_name='mapping')

In [ ]:
jbc_mapping = jbc_mapping_raw.melt(id_vars=['Region', 'Sub-region/Country'], value_name='country')

In [ ]:
jbc_mapping.to_csv('jbc_mapping2.csv')

In [ ]:
jbc_mapping.variable.unique()

In [ ]:
jbc_sort = pd.read_excel('//petroineos.local/DFS/Profiles/syedahmad/Desktop/country mapping.xlsx', sheet_name='jbc mapping')
jbc_sort.sort_values(by=['Region', 'Sub-region/Country', 'country'], inplace=True)
jbc_sort.drop_duplicates(inplace=True)

In [ ]:
jbc_sort.to_csv('jbc_mapping3.csv')

In [ ]:
# jbc_mapping.drop(columns='variable', inplace=True)
# jbc_mapping = jbc_mapping[jbc_mapping['country'].notnull()]
# jbc_mapping.sort_values(by=['Region', 'Sub-region/Country', 'country'], inplace=True)

In [ ]:
# jbc_mapping.to_csv('jbc_mapping.csv')

### ---------------------------- Gas Prices -----------------------------------------

In [ ]:
gas_spot = pd.read_excel('gas_prices/ttf_heren.xlsx', sheet_name='Sheet1', header=1)
gas_spot.columns = ['day', 'eur/mwh']
gas_spot.set_index('day', inplace=True)

In [ ]:
gas_spot_monthly = gas_spot.resample('MS').mean()

In [ ]:
gas_spot_monthly.tail()

In [ ]:
gas_forward_curve = pd.read_excel('gas_prices/TTF.xlsx', sheet_name='curve', header=0)
gas_forward_curve.rename(columns={'month':'day'}, inplace=True)
gas_forward_curve.set_index('day', inplace=True)

In [ ]:
gas_forward_curve.head()

In [ ]:
# gas_spot_monthly.to_excel('gas_prices/ttf monthly spot heren.xlsx', sheet_name='spot_monthly')

In [ ]:
gas_combined = pd.concat([gas_spot_monthly, gas_forward_curve], axis=0, sort=False)

In [ ]:
eur_usd_daily = pd.read_excel('gas_prices/usd_eur.xlsx', header=1)
eur_usd_daily.columns = ['day', 'usd per eur']
eur_usd_daily.set_index('day', inplace=True)

In [ ]:
eur_usd_monthly = eur_usd_daily.resample('MS').mean()
# eur_usd_monthly.rename(index={'day':'month'}, inplace=True)

In [ ]:
eur_usd_monthly.head()

In [ ]:
# combined_ttf = pd.read_excel('TTF.xlsx', sheet_name='combined').set_index('Month')

In [ ]:
ttf_fx = gas_combined.merge(eur_usd_monthly, how='left', left_index=True, right_index=True)
ttf_fx['usd per eur'].fillna(method='ffill', inplace=True)
ttf_fx['usd/MWH'] = ttf_fx['eur/mwh']*ttf_fx['usd per eur']
ttf_fx['gas cost (0.06) usd/barrel'] = ttf_fx['usd/MWH']*0.06
ttf_fx['gas cost (0.04) usd/barrel'] = ttf_fx['usd/MWH']*0.04
ttf_fx.reset_index(inplace=True)
ttf_fx.rename(columns={'day':'month'}, inplace=True)
ttf_fx.set_index('month', inplace=True)

In [ ]:
# ttf_fx.to_csv('ttf_fx_rate.csv')

In [ ]:
ttf_fx.head()

In [ ]:
gas_combined.reset_index(inplace=True)
gas_combined.rename(columns={'day':'month'}, inplace=True)

In [ ]:
seasonal_gas_price = get_pivot(gas_combined)

In [ ]:
seasonal_gas_price.to_excel('seaonal_gas_curve.xlsx')

### -------------------- cleaning woodmac data ---------------------------

In [32]:
def date_cleaner(df):
    df['date_str'] = df['Date (dd/mm/yyyy)'].astype(str)
    df['month'] = pd.to_datetime(df['date_str'],infer_datetime_format=False, format='%Y-%d-%m')
    df.drop(columns=['Date (dd/mm/yyyy)','date_str'], inplace=True)
    return df

In [36]:
woodmac_prices = pd.read_excel('refining margins/woodmac prices raw.xlsx')
woodmac_prices_clean = date_cleaner(woodmac_prices)

In [37]:
woodmac_ref = pd.read_excel('woodmac data/woodmac_refinery_ops.xlsx')
woodmac_ref_clean = date_cleaner(woodmac_ref)

In [ ]:
woodmac_bal = pd.read_excel('balances/wdmc_raw_balance.xlsx')
woodmac_bal_clean = date_cleaner(woodmac_bal)

In [ ]:
# woodmac_ref_clean.to_excel('refining margins/woodmac_refinery_ops_cln.xlsx')

In [ ]:
# woodmac_prices.to_csv('woodmac_prices_cleaned.csv')

### --------------- woodmac seasonal margins ----------------

In [ ]:
woodmac_prices.Region.unique()

In [ ]:
woodmac_prices.Metric.unique()

In [ ]:
global_refining_margins = woodmac_prices[woodmac_prices['Metric']=='Refining Margins'].copy()
nwe_margins = global_refining_margins[global_refining_margins['Region']=='NWE'].copy()
med_margins = global_refining_margins[global_refining_margins['Region']=='Mediterranean'].copy()

In [ ]:
med_margins_pivot = med_margins.pivot(index='month', columns='Product', values='Value')
nwe_margins_pivot = nwe_margins.pivot(index='month', columns='Product', values='Value')

In [ ]:
nwe_margins_pivot_filter = nwe_margins_pivot[['Brent HCU', 'Brent FCC', 'Urals HCU', 'Urals FCC']].copy()

In [ ]:
nwe_gas = nwe_margins_pivot_filter.merge(ttf_fx[['gas cost (0.06) usd/barrel', 'gas cost (0.04) usd/barrel']], how='left', left_index=True, right_index=True)

In [ ]:
nwe_gas['Brent HCU - with gas cost'] = nwe_gas['Brent HCU'] - nwe_gas['gas cost (0.06) usd/barrel']
nwe_gas['Brent FCC - with gas cost'] = nwe_gas['Brent FCC'] - nwe_gas['gas cost (0.04) usd/barrel']
nwe_gas['Urals HCU - with gas cost'] = nwe_gas['Urals HCU'] - nwe_gas['gas cost (0.06) usd/barrel']
nwe_gas['Urals FCC - with gas cost'] = nwe_gas['Urals FCC'] - nwe_gas['gas cost (0.04) usd/barrel']

In [ ]:
nwe_gas

In [ ]:
month_table = brent_hcu[['month', 'month_name']].copy()
month_table = month_table.drop_duplicates()

In [ ]:
month_mapping = month_table.set_index('month')

In [ ]:
brent_hcu = nwe_gas[['Brent HCU - with gas cost']].copy()
brent_hcu.reset_index(inplace=True)
brent_hcu['year']=brent_hcu.month.dt.year
# brent_hcu['month_name']=brent_hcu.month.dt.month_name(locale = 'English')
brent_hcu['month']=brent_hcu.month.dt.month
brent_hcu_pivot = brent_hcu.pivot(index='month', columns='year')
brent_hcu_pivot.reset_index(inplace=True)
brent_hcu_pivot['month'] = brent_hcu_pivot['month'].apply(lambda x: calendar.month_abbr[x])

In [ ]:
brent_hcu_pivot

In [ ]:
brent_hcu_pivot.to_csv('brent_hcu.csv')

In [ ]:
def get_pivot(df):
    df_new = df.copy()
#     df_new.reset_index(inplace=True)
    df_new['year']=df_new.month.dt.year
    df_new['month']=df_new.month.dt.month
    df_new = df_new.pivot(index='month', columns='year')
    df_new.reset_index(inplace=True)
    df_new['month'] = df_new['month'].apply(lambda x: calendar.month_abbr[x])
    return df_new

In [ ]:
nwe_gas.columns

In [ ]:
brent_hcu_final = get_pivot(nwe_gas[['Brent HCU']])
brent_fcc_final = get_pivot(nwe_gas[['Brent FCC']])
urals_hcu_final = get_pivot(nwe_gas[['Urals HCU']])
urals_fcc_final = get_pivot(nwe_gas[['Urals FCC']])
brent_fcc_gas = get_pivot(nwe_gas[['Brent FCC - with gas cost']])
urals_hcu_gas = get_pivot(nwe_gas[['Urals HCU - with gas cost']])
urals_fcc_gas = get_pivot(nwe_gas[['Urals FCC - with gas cost']])

In [ ]:
urals_fcc_gas

In [ ]:
writer = pd.ExcelWriter('nwe_seasonal_margins.xlsx', engine='xlsxwriter')

In [ ]:
brent_hcu_final.to_excel(writer, sheet_name='Brent HCU')
brent_fcc_final.to_excel(writer, sheet_name='Brent FCC')
urals_hcu_final.to_excel(writer, sheet_name='Urals HCU')
urals_fcc_final.to_excel(writer, sheet_name='Urals FCC')
brent_hcu_pivot.to_excel(writer, sheet_name='Brent HCU - with gas cost')
brent_fcc_gas.to_excel(writer, sheet_name='Brent FCC - with gas cost')
urals_hcu_gas.to_excel(writer, sheet_name='Urals HCU - with gas cost')
urals_fcc_gas.to_excel(writer, sheet_name='Urals FCC - with gas cost')

In [ ]:
writer.save()

### -------------med margins-----------------

In [ ]:
med_margins_pivot.head()

In [ ]:
med_margins_pivot_filter = med_margins_pivot[['Basrah Light HCU', 'Basrah Light FCC', 'Urals HCU', 'Urals FCC']].copy()
med_gas = med_margins_pivot_filter.merge(ttf_fx[['gas cost (0.06) usd/barrel', 'gas cost (0.04) usd/barrel']], how='left', left_index=True, right_index=True)

In [ ]:
med_gas['Basrah Light HCU - with gas cost'] = med_gas['Basrah Light HCU'] - med_gas['gas cost (0.06) usd/barrel']
med_gas['Basrah Light FCC - with gas cost'] = med_gas['Basrah Light FCC'] - med_gas['gas cost (0.04) usd/barrel']
med_gas['Urals HCU - with gas cost'] = med_gas['Urals HCU'] - med_gas['gas cost (0.06) usd/barrel']
med_gas['Urals FCC - with gas cost'] = med_gas['Urals FCC'] - med_gas['gas cost (0.04) usd/barrel']

In [ ]:
med_gas.head()

In [ ]:
bas_hcu_med = get_pivot(med_gas[['Basrah Light HCU']])
bas_fcc_med = get_pivot(med_gas[['Basrah Light FCC']])
urals_hcu_med = get_pivot(med_gas[['Urals HCU']])
urals_fcc_med = get_pivot(med_gas[['Urals FCC']])
bas_hcu_gas_med = get_pivot(med_gas[['Basrah Light HCU - with gas cost']])
bas_fcc_gas_med = get_pivot(med_gas[['Basrah Light FCC - with gas cost']])
urals_hcu_gas_med = get_pivot(med_gas[['Urals HCU - with gas cost']])
urals_fcc_gas_med = get_pivot(med_gas[['Urals FCC - with gas cost']])

In [ ]:
writer = pd.ExcelWriter('med_seasonal_margins_new.xlsx', engine='xlsxwriter')
bas_hcu_med.to_excel(writer, sheet_name='Basrah Light HCU')
bas_fcc_med.to_excel(writer, sheet_name='Basrah Light FCC')
urals_hcu_med.to_excel(writer, sheet_name='Urals HCU')
urals_fcc_med.to_excel(writer, sheet_name='Urals FCC')
bas_hcu_gas_med.to_excel(writer, sheet_name='Basrah HCU - gas cost')
bas_fcc_gas_med.to_excel(writer, sheet_name='Basrah FCC - gas cost')
urals_hcu_gas_med.to_excel(writer, sheet_name='Urals HCU - gas cost')
urals_fcc_gas_med.to_excel(writer, sheet_name='Urals FCC - gas cost')
writer.save()

#### ------------------wdmc refinery data ---------------------

In [ ]:
ref_capacity = woodmac_ref_clean[woodmac_ref_clean['Metric']=='CDU Capacity']

In [ ]:
ref_capacity.Region.unique()

In [ ]:
ref_capacity_rus = ref_capacity[ref_capacity.Region=='Russia and the Caspian'].copy()
ref_capacity_na = ref_capacity[ref_capacity.Region=='North America'].copy()
ref_capacity_ap = ref_capacity[ref_capacity.Region=='Asia Pacific'].copy()
ref_capacity_eu = ref_capacity[ref_capacity.Region=='Europe'].copy()
ref_capacity_latam = ref_capacity[ref_capacity.Region=='Latin America'].copy()
ref_capacity_af = ref_capacity[ref_capacity.Region=='Africa'].copy()
ref_capacity_meg = ref_capacity[ref_capacity.Region=='Middle East'].copy()

In [ ]:
ref_capacity_na.Location.unique()
us_msk = ref_capacity_na.Location!='United States'
ref_capacity_na = ref_capacity_na[us_msk].copy()

In [ ]:
def ref_pivot(df):
    df_new = df.groupby(['Region', 'month']).sum()
    df_new.reset_index(inplace=True)
    df_new['year']=df_new.month.dt.year
    df_new['month']=df_new.month.dt.month
    df_new = df_new.pivot(index='month', columns='year', values='Value')
    df_new.reset_index(inplace=True)
    df_new['month'] = df_new['month'].apply(lambda x: calendar.month_abbr[x])
    return df_new

In [ ]:
rus_pivot = ref_pivot(ref_capacity_rus)
na_pivot = ref_pivot(ref_capacity_na)
ap_pivot = ref_pivot(ref_capacity_ap)
eu_pivot = ref_pivot(ref_capacity_eu)
latam_pivot = ref_pivot(ref_capacity_latam)
af_pivot = ref_pivot(ref_capacity_af)
meg_pivot = ref_pivot(ref_capacity_meg)

In [ ]:
writer = pd.ExcelWriter('refining margins/refining_seasonal_capacities_ex_us.xlsx', engine='xlsxwriter')
rus_pivot.to_excel(writer, sheet_name='Russia')
na_pivot.to_excel(writer, sheet_name='north america')
ap_pivot.to_excel(writer, sheet_name='asia pacific')
eu_pivot.to_excel(writer, sheet_name='europe')
latam_pivot.to_excel(writer, sheet_name='latam')
af_pivot.to_excel(writer, sheet_name='africa')
meg_pivot.to_excel(writer, sheet_name='middle east')
writer.save()

### ------------------------ woodmac inventories -------------------------------

In [ ]:
woodmac_bal_clean.Metric.unique()

In [ ]:
stocks = woodmac_bal_clean[woodmac_bal_clean['Metric']=='Stocks (Absolute)']

In [ ]:
stocks['Product Name'].unique()

In [ ]:
lpg = stocks[stocks['Product Name']=='LPG'].copy()
nap = stocks[stocks['Product Name']=='Naphtha'].copy()
gas = stocks[stocks['Product Name']=='Gasoline'].copy()
jet = stocks[stocks['Product Name']=='Jet/Other Kerosene'].copy()
dist = stocks[stocks['Product Name']=='Diesel/Gasoil'].copy()
fo = stocks[stocks['Product Name']=='Fuel Oil'].copy()

In [ ]:
def stock_pivot(df):
    df_new = df.groupby(['month']).sum()
    df_new.reset_index(inplace=True)
    df_new['year']=df_new.month.dt.year
    df_new['month']=df_new.month.dt.month
    df_new = df_new.pivot(index='month', columns='year', values='Value')
    df_new.reset_index(inplace=True)
    df_new['month'] = df_new['month'].apply(lambda x: calendar.month_abbr[x])
    return df_new

In [ ]:
lpg_pivot = stock_pivot(lpg)
nap_pivot = stock_pivot(nap)
gas_pivot = stock_pivot(gas)
jet_pivot = stock_pivot(jet)
dist_pivot = stock_pivot(dist)
fo_pivot = stock_pivot(fo)

In [ ]:
writer = pd.ExcelWriter('balances/wdmc_seasonal_stocks.xlsx', engine='xlsxwriter')
lpg_pivot.to_excel(writer, sheet_name='lpg')
nap_pivot.to_excel(writer, sheet_name='naphtha')
gas_pivot.to_excel(writer, sheet_name='gasoline')
jet_pivot.to_excel(writer, sheet_name='jet')
dist_pivot.to_excel(writer, sheet_name='distillate')
fo_pivot.to_excel(writer, sheet_name='fuel_oil')
writer.save()

### ---------- rystad data --------------------

In [25]:
# woz = pd.read_excel('balances/rystad WoZ products demand.xlsx')
# eoz = pd.read_excel('balances/rystad prod demand EoZ.xlsx')
# demand_global = pd.read_excel('balances/rystad prod demand global.xlsx')
prod = pd.read_excel('balances/rystad_production.xlsx')
prod['Year'] = prod['Year'].astype(str)
prod['month'] = prod[['Month', 'Year']].agg('-'.join, axis=1)

In [26]:
def rystad_pivot(product, df):
    df_new = df[['month', product]].copy()
    df_new['month']=pd.to_datetime(df_new['month'])
    df_new['year']=df_new.month.dt.year
    df_new['month']=df_new.month.dt.month
    df_new = df_new.pivot(index='month', columns='year')
    df_new.reset_index(inplace=True)
    df_new['month'] = df_new['month'].apply(lambda x: calendar.month_abbr[x])
    return df_new

In [ ]:
lpg_demand = rystad_pivot('LPG', woz)
nap_demand = rystad_pivot('Naphtha', woz)
gas_demand = rystad_pivot('Gasoline', woz) 
jet_demand = rystad_pivot('Jet fuel and kerosene', woz)
dist_demand = rystad_pivot('Distillates', woz)
fo_demand = rystad_pivot('Fuel oil', woz)

In [ ]:
writer = pd.ExcelWriter('balances/rystad_seasonal_demand_woz.xlsx', engine='xlsxwriter')
lpg_demand.to_excel(writer, sheet_name='lpg')
nap_demand.to_excel(writer, sheet_name='naphtha')
gas_demand.to_excel(writer, sheet_name='gasoline')
jet_demand.to_excel(writer, sheet_name='jet')
dist_demand.to_excel(writer, sheet_name='distillate')
fo_demand.to_excel(writer, sheet_name='fuel_oil')
writer.save()

------------------ east of suez -------------------

In [ ]:
lpg_demand = rystad_pivot('LPG', eoz)
nap_demand = rystad_pivot('Naphtha', eoz)
gas_demand = rystad_pivot('Gasoline', eoz) 
jet_demand = rystad_pivot('Jet fuel and kerosene', eoz)
dist_demand = rystad_pivot('Distillates', eoz)
fo_demand = rystad_pivot('Fuel oil', eoz)

In [ ]:
writer = pd.ExcelWriter('balances/rystad_seasonal_demand_eoz.xlsx', engine='xlsxwriter')
lpg_demand.to_excel(writer, sheet_name='lpg')
nap_demand.to_excel(writer, sheet_name='naphtha')
gas_demand.to_excel(writer, sheet_name='gasoline')
jet_demand.to_excel(writer, sheet_name='jet')
dist_demand.to_excel(writer, sheet_name='distillate')
fo_demand.to_excel(writer, sheet_name='fuel_oil')
writer.save()

------------------ production -------------------

In [5]:
prod.columns

Index(['Year', 'Month', 'Persian Gulf', 'Mediterranean', 'West Africa',
       'Latin America', 'PADD 1: East Cost', 'PADD 2: Midwest',
       'PADD 3: Gulf Coast (onshore)', 'PADD 3: Gulf of Mexico (offshore)',
       'PADD 4: Rocky Mountain', 'PADD 5: West Coast', 'North Sea',
       'Russia (domestic)'],
      dtype='object')

In [27]:
prod_us = prod[['month','PADD 1: East Cost', 'PADD 2: Midwest',
       'PADD 3: Gulf Coast (onshore)', 'PADD 3: Gulf of Mexico (offshore)',
       'PADD 4: Rocky Mountain', 'PADD 5: West Coast']]

In [28]:
prod_us['usa'] = prod_us.sum(axis=1)

C:\conda\envs\lng\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [29]:
prod_us_total = prod_us[['month', 'usa']].copy()

In [30]:
waf = rystad_pivot('West Africa', prod)
north_sea = rystad_pivot('North Sea', prod)
med = rystad_pivot('Mediterranean', prod)
latam = rystad_pivot('Latin America', prod)
usa = rystad_pivot('usa', prod_us_total)

In [31]:
writer = pd.ExcelWriter('balances/rystad_seasonal_production.xlsx', engine='xlsxwriter')
waf.to_excel(writer, sheet_name='West africa')
north_sea.to_excel(writer, sheet_name='North sea')
med.to_excel(writer, sheet_name='Med')
latam.to_excel(writer, sheet_name='Latam')
usa.to_excel(writer, sheet_name='USA')
writer.save()

In [ ]:
usa.to_excel('balances/usa_seasonal_production.xlsx')

### ------------------------ price forecasts -------------------------------

In [ ]:
brent_daily_hist = pd.read_excel('price_forecasts/brent.xlsx', header=1)
brent_daily_hist.columns=['day', 'usd/bbl']
# brent_daily_hist.day = pd.to_datetime(brent_daily_hist.day)
brent_daily_hist.set_index('day', inplace=True)
brent_mnth_hist = brent_daily_hist.resample('MS').mean()

In [ ]:
def daily_monthly_price(filepath):
    df = pd.read_excel(filepath, header=1)
    df.columns=['day', 'usd/bbl']
    df.set_index('day', inplace=True)
    df_new = df.resample('MS').mean()
    return df_new

In [ ]:
brent_mnth_hist = daily_monthly_price('price_forecasts/brent.xlsx')
dtd_mnth_hist = daily_monthly_price('price_forecasts/dtd_raw.xlsx')

In [ ]:
# brent_mnth_hist.loc['2021-01-01':].to_excel('price_forecasts/brent_monthly.xlsx')

In [ ]:
dtd_mnth_hist.loc['2021-01-01':].to_excel('price_forecasts/dtd_monthly.xlsx')